# Projeto 1 - Ciência dos Dados

Nome: Amanda Carmo

Nome: Vitor Miada

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___

## Introdução



Com a crescente dos E-Sports, os tópicos sobre jogos e campeonatos vem cada vez sendo mais significantes. Um dos cenários mais desenvolvidos, é o competitivo de "Counter Strike"(CS).
CS, é um jogo First Person Shooter (FPS), de agilidade e estratégia, onde 10 jogadores divididos em 2 times, 5 versus 5 se enfrentam. Diante desse cenário, um dos principais times, é o "MIBR", uma organização brasileira.


O objetivo deste projeto pauta-se na criação de um classificador que analise a opinião daqueles que acompanham o time de Counter-Strike, MIBR.
Para tal fim, será feita a análise probabilística de um tweet ser relevante, de acordo com as classificações do grupo, dadas as palavras presentes nele. O meio que irá ser usado é o princípio do algoritimo de Naive Bayes.


___
Carregando algumas bibliotecas:

In [37]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [38]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/home/borg/Documents/Insper/Cdados/p1_dados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [39]:
dados = pd.read_excel('mibr.xlsx', sheet_name='Treinamento')
dados.head()

,Classificacao,Treinamento
0,2,"@smooyacs @mibr tomar no seu cu, tem quanros t..."
1,3,mibr ta igual o cruzeiro kkkkkkkkkkkkkkkkkkkkk...
2,3,@maycon_design @mibr f brasileirinho eu sou a ...
3,3,"@arthurmelo22_ @mibr @chaosec @envy tem boltz,..."
4,2,@mibr @chaosec @envy piada kkkkkkkkkkkkkkkkkkk...


In [40]:
dados_teste = pd.read_excel('mibr.xlsx', sheet_name='Teste')

#renomeando coluna de teste a fim de diferenciar classificação manual e da máquina.
dados_teste.rename(columns={'Unnamed: 1':'Classificação manual'}, inplace=True) 
dados_teste.head()

,Classificação,Teste
0,1,cs é um bom exemplo do quanto a experiencia in...
1,2,"acho que o time precisa de uma pausa, tentar p..."
2,3,"mano, o que ta acontecendo com a mibr"
3,3,@rafael_loppes1 @mibr @abcfc eu tava acompanha...
4,3,mibr decepcionou hoje :/ pelo menos to feliz p...


___
# Classificador automático de sentimento




O produto a ser analisado, é de um time profissional brasileiro de "Counter Strike Global Offensive" (CSGO), chamado MIBR. Foram considerados relevantes, tweets que mostrem satisfação ou insatisfação sobre a organização do time. 

Foram feitas três classificações para a análise dos tweets:

#### - Negativos: **"N"**
Comentários críticos ao time ou pejorativos (dados a derrotas, por exemplo ou performance de jogo);

#### - Positivos: **"P"**
Elogios e apoio de torcedores, ressalvas a respeito da perfrmance em dado campeonato, por exemplo;

#### - Irrelevantes: **"I"**
Considera-se irrelevantes aqueles que não agragaram-se às classificações descritas anteriormente, por exemplo, piadas, avisos a respeito do time, etc.

### Limpezas dos tweets:

In [41]:
import re 

def cleanup(text):
    
    #Função para substituir pontos por espaços.    
    punctuation = '[!\-.:?;•,]' #Pontuações que vão ser excluídas.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    
    return text_subbed

#### Assinala-se que as classificações são variáveis qualitativas

In [42]:
dados['Classificacao'] = dados['Classificacao'].astype('category')

In [43]:
dados['Classificacao'].cat.categories = ['P', 'I', 'N']

# Visualizando o resultado
dados['Classificacao'].cat.categories

Index(['P', 'I', 'N'], dtype='object')

In [44]:
dados['Classificacao'].head(5)

0    I
1    N
2    N
3    N
4    I
Name: Classificacao, dtype: category
Categories (3, object): [P, I, N]

In [45]:
dados_teste['Classificação'] = dados_teste['Classificação'].astype('category')

dados_teste['Classificação'].cat.categories = ['P', 'I', 'N']
# Visualizando o resultado
dados_teste['Classificação'].cat.categories

Index(['P', 'I', 'N'], dtype='object')

In [46]:
dados_teste['Classificação'].head(5)

0    P
1    I
2    N
3    N
4    N
Name: Classificação, dtype: category
Categories (3, object): [P, I, N]

In [47]:
dados_teste

,Classificação,Teste
0,P,cs é um bom exemplo do quanto a experiencia in...
1,I,"acho que o time precisa de uma pausa, tentar p..."
2,N,"mano, o que ta acontecendo com a mibr"
3,N,@rafael_loppes1 @mibr @abcfc eu tava acompanha...
4,N,mibr decepcionou hoje :/ pelo menos to feliz p...
...,...,...
313,N,"caralho mano, \nn vim criticar, mas pqp\na mib..."
314,I,@fly_vitor11 @smooyacs @mibr congrats o caralh...
315,I,"@mibr taco e mey, corram enquanto é tempo, por..."
316,N,@bida @mibr @flashpoint @chaosec ex time em at...


___
## Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [48]:
# P(P), P(I), P(N)
dados.Classificacao.value_counts(True)

I    0.5550
N    0.3075
P    0.1375
Name: Classificacao, dtype: float64

### <font color='#005675'>Funções para frequências</font>

Para obter-se a frequência relativa, conforme visto na aula, precisa-se inicialmente do comando `split()` e em seguida tranformar os dados em uma série do pandas, com `pd.Series()`, tal como foi usado a seguir.

Para uma limpeza adicional, foi adicionado às funções `re.sub(r'http\S+', '', tw_completo)`, o qual remove os http que podem intervir na classificação.

#### <font color='#005675'>Frequência Relativa</font>

In [49]:
def Frequencia_Relativa(cat):
    #tira espaços
    tweet = ' '.join(str(cat) for cat in dados[dados.Classificacao==cat].Treinamento)
    
    #Extra: remoção dos "http" para não enviezar o classificador.
    remove_http = re.sub(r'http\S+', '', tweet)
    text_clean = cleanup(remove_http)
    
    #split() e pd.Series()
    serie_dados = text_clean.split()
    freq_rel = pd.Series(serie_dados).value_counts(True)
    
    return freq_rel

In [50]:
# P(palavra|P)
Frequencia_Relativa('P').head()


e       0.027061
que     0.025802
a       0.023285
de      0.022026
mibr    0.020768
dtype: float64

In [51]:
# P(palavra|N)
Frequencia_Relativa('N').head()

@mibr    0.030651
mibr     0.028948
que      0.025543
o        0.024691
e        0.023414
dtype: float64

In [52]:
# P(palavra|I)
Frequencia_Relativa('I').head()

o        0.031155
@mibr    0.028775
a        0.024232
mibr     0.023799
que      0.022068
dtype: float64

In [53]:
dados_P = dados[dados.Classificacao == 'P'].copy()
dados_P.head(5)

,Classificacao,Treinamento
10,P,"@vrf_jt @mibr @chaosec @envy assim espero, são..."
13,P,@fafonaves @roque_mn lembra do flamengo? fase ...
14,P,"@mibr @chaosec @envy por favor, chamem o @zorl..."
21,P,@chaosec @mibr @geng @flashpoint @joshnissan @...
39,P,rt @guizaokemen: meu único pedido de aniversár...


In [54]:
dados_I = dados[dados.Classificacao == 'I'].copy()
dados_I.head(5)

,Classificacao,Treinamento
0,I,"@smooyacs @mibr tomar no seu cu, tem quanros t..."
4,I,@mibr @chaosec @envy piada kkkkkkkkkkkkkkkkkkk...
5,I,@eldereti @mibr @chaosec @envy obrigado china
6,I,@eldereti @bida @mibr @flashpoint @chaosec tá ...
12,I,@mibr @chaosec @envy fechem as portas..


In [55]:
dados_N = dados[dados.Classificacao == 'N'].copy()
dados_N.head(5)

,Classificacao,Treinamento
1,N,mibr ta igual o cruzeiro kkkkkkkkkkkkkkkkkkkkk...
2,N,@maycon_design @mibr f brasileirinho eu sou a ...
3,N,"@arthurmelo22_ @mibr @chaosec @envy tem boltz,..."
7,N,@gio_fps mas esperar que a comunidade motive e...
8,N,@gaules o chaos não existia na cenário do mund...


#### <font color='#005675'>Frequência Absoluta</font>

In [113]:
def Frequencia_Absoluta(cat):
    
    tweet = ' '.join(str(cat) for cat in dados[dados.Classificacao==cat].Treinamento)
    remove_http = re.sub(r'http\S+', '', tweet)
    
    text_clean = cleanup(remove_http)
    serie_dados = text_clean.split()
    freq_abs = pd.Series(serie_dados).value_counts()
    
    
    return freq_abs

In [114]:
Frequencia_Absoluta('P').head()

e       43
que     41
a       37
de      35
mibr    33
dtype: int64

In [115]:
Frequencia_Absoluta('N').head()

@mibr    72
mibr     68
que      60
o        58
e        55
dtype: int64

In [116]:
Frequencia_Absoluta('I').head()

o        144
@mibr    133
a        112
mibr     110
que      102
dtype: int64

### <font color='#005675'>Probabilidades</font>

In [117]:
def clean_extra(cat):
    tweet = ' '.join(str(cat) for cat in dados[dados.Classificacao==cat].Treinamento)
    remove_http = re.sub(r'http\S+','', tweet)    
    text_clean = cleanup(remove_http)
    clean_tweet = text_clean.split() 
    #serie_tweet = pd.Series(clean_tweet)

    return clean_tweet

tweet_I = clean_extra('I')
tweet_P = clean_extra('P')
tweet_N = clean_extra('N')



print("Quantidade de palavras em tweets irrelevantes:")
print(len(tweet_I))
print('')
print("Quantidade de palavras em tweets positivos:")
print(len(tweet_P))
print('')
print("Quantidade de palavras em tweets negativos:")
print(len(tweet_N))

Quantidade de palavras em tweets irrelevantes:
4622

Quantidade de palavras em tweets positivos:
1589

Quantidade de palavras em tweets negativos:
2349


In [118]:
all_tweets = tweet_I + tweet_N + tweet_P

In [119]:
all_quant = len(all_tweets)
all_quant

8560

In [120]:
lista = []
for i in all_tweets:
    if i not in lista:
        lista.append(i)
lista

['@smooyacs',
 '@mibr',
 'tomar',
 'no',
 'seu',
 'cu',
 'tem',
 'quanros',
 'títulos',
 'kkkkkkkkkk',
 '🤣\U0001f970',
 '@chaosec',
 '@envy',
 'piada',
 'kkkkkkkkkkkkkkkkkkkkkkkkkkkkkkk',
 'nao',
 'da',
 'mais',
 'cara',
 'pra',
 'mim',
 'chega',
 'aguento',
 '@eldereti',
 'obrigado',
 'china',
 '@bida',
 '@flashpoint',
 'tá',
 'desculpado',
 'meu',
 'amigo',
 'fechem',
 'as',
 'portas',
 'rt',
 '@betwayesportsbr',
 'que',
 'espeto',
 'do',
 '@kngvito',
 'hoje',
 '2',
 'pelo',
 'preço',
 'de',
 'um',
 '1',
 'na',
 '@alvaroyuri10',
 'so',
 'camp',
 'man',
 'só',
 '@mibr_picked_inf',
 'sei',
 'pq',
 'chamam',
 'chelo',
 'ou',
 'boltz',
 'mas',
 'ficam',
 'com',
 'kng',
 'taco',
 'e',
 'meyern',
 '@vitorsccp2005',
 '@costa91manoel',
 'eu',
 'sigo',
 'como',
 'referência',
 'os',
 'times',
 '#8',
 'até',
 'o',
 '#1',
 'hltv',
 'eles',
 'nunca',
 'tiveram',
 'tanta',
 'competição',
 'para',
 'pegar',
 'primeiro',
 'lugar',
 'hj',
 'em',
 'dia',
 'antes',
 'eram',
 '"resto"',
 'tlgd',
 'por'

In [121]:
palavras_existentes = len(lista)

In [122]:
def prob_category(cat):
    tweet = clean_extra(cat)
    prob = len(tweet)/all_quant
    return prob

In [123]:
prob_I = prob_category('I')
prob_I

0.5399532710280374

In [124]:
prob_P = prob_category('P')
prob_P

0.18563084112149533

In [125]:
prob_N = prob_category('N')
prob_N

0.2744158878504673

In [126]:
print('')
print("Probabilidade Total (checa se dá 1):")
print(prob_P + prob_N + prob_I)


Probabilidade Total (checa se dá 1):
1.0


___
## Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [127]:
try:
    Freqi = Frequencia_Absoluta('I')
    Freqn = Frequencia_Absoluta('N')
    Freqp = Frequencia_Absoluta('P')
except: 
    pass


pP = np.sum(Freqp)
pN = np.sum(Freqn)
pI = np.sum(Freqi)
pT = pP + pN + pI



In [128]:
pT

8560

In [129]:
e = 1e6

In [130]:
alpha = 1

In [183]:
#P(palavra|Positivo) = P(Positivo U_ palavra)/P(Positivo)


def smoothing(Freq, pClas, cat):
    Dic_probs = {}
    
    tweet = clean_extra(cat)
    prob = len(tweet)/all_quant
    
    for tt in dados_teste['Teste']:
        tt = cleanup(tt)
        prob_tt = 1
        palavra_tt = tt.split()
        for palavra_teste in palavra_tt:
            s = 0 # frequencia da palavra no Positivo
            p = 0
            if palavra_teste in Freqp:
                s = Freq[palavra_teste]   #pegando qtas vezes aparece a palavra
            p = (s + alpha)/(pClas + palavras_existentes)
            prob_tt = prob_tt * p
    
        prob_fin = prob_tt * prob
        Dic_probs[tt] = prob_fin
    return Dic_probs
    

In [184]:
#Positivo = smoothing(Freqp, pP, 'P')

In [185]:
#Indiferente = smoothing(Freqi, pI, 'I')

In [177]:
#Negativo = smoothing(Freqp, pN, 'N')

In [182]:
Positivo = {}


for tt in dados_teste['Teste']:
    tt = cleanup(tt)
    prob_tt = 1
    palavra_tt = tt.split()
    for palavra_teste in palavra_tt:
        s = 0 # frequencia da palavra no Positivo
        p = 0
        if palavra_teste in Freqp:
            s = Freqp[palavra_teste]   #pegando qtas vezes aparece a palavra
        p = (s + alpha)/(pP + palavras_existentes)
        prob_tt = prob_tt * p
    
    prob_fin = prob_tt * prob_category('P')
    Positivo[tt] = prob_fin
        
Positivo

{'cs é um bom exemplo do quanto a experiencia internacional faz diferença  lg/sk/mibr tiveram que se mudar do brasil pra atingir o topo  apesar da contínua queda de desempenho  mas ninguém permanece por muito tempo no topo em esporte nenhum ': 1.5691114401900081e-121,
 'acho que o time precisa de uma pausa  tentar por a cabeça no lugar e entender o novo meio de se jogar csgo  pra depois voltarem a jogar juntos  cold viu que o time estagnou e vazou fora pra faze  mibr precisa se reinventar mas precisam estar com a mente no lugar': 5.0750774332161537e-149,
 'mano  o que ta acontecendo com a mibr': 4.0074344912234782e-21,
 '@rafael_loppes1 @mibr @abcfc eu tava acompanhando a fúria  assisti aquela md5 que eles perderam a final para a geng \n\ntaco  fallen  fer e coldzera eram a base do time  cold saiu e só ficou esses 3  acho que se um desses sair o time acaba  hoje tá só o remendo ': 2.916705264134217e-148,
 'mibr decepcionou hoje  / pelo menos to feliz por motivos óbvios': 3.986986622620

In [178]:
#P(palavra|Negativo) = P(Negativo U_ palavra)/P(Negativo)

Negativo = {}


for tt in dados_teste['Teste']:
    tt = cleanup(tt)
    prob_tt = 1
    palavra_tt = tt.split()
    for palavra_teste in palavra_tt:
        s = 0 # frequencia da palavra no Negativo
        p = 0
        if palavra_teste in Freqn:
            s = Freqn[palavra_teste]   #pegando qtas vezes aparece a palavra
        p = (s + alpha)/(pN + palavras_existentes)
        prob_tt = prob_tt * p
    
    prob_fin = prob_tt * prob_category('N')
    Negativo[tt] = prob_fin
        

Negativo

{'cs é um bom exemplo do quanto a experiencia internacional faz diferença  lg/sk/mibr tiveram que se mudar do brasil pra atingir o topo  apesar da contínua queda de desempenho  mas ninguém permanece por muito tempo no topo em esporte nenhum ': 7.4548156293558324e-121,
 'acho que o time precisa de uma pausa  tentar por a cabeça no lugar e entender o novo meio de se jogar csgo  pra depois voltarem a jogar juntos  cold viu que o time estagnou e vazou fora pra faze  mibr precisa se reinventar mas precisam estar com a mente no lugar': 1.0088533684840013e-147,
 'mano  o que ta acontecendo com a mibr': 2.2308807516356433e-20,
 '@rafael_loppes1 @mibr @abcfc eu tava acompanhando a fúria  assisti aquela md5 que eles perderam a final para a geng \n\ntaco  fallen  fer e coldzera eram a base do time  cold saiu e só ficou esses 3  acho que se um desses sair o time acaba  hoje tá só o remendo ': 2.6753499947585578e-145,
 'mibr decepcionou hoje  / pelo menos to feliz por motivos óbvios': 7.04853119216

In [179]:
#P(palavra|Indiferente) = P(Indiferente U_ palavra)/P(Indiferente)

Indiferente = {}


for tt in dados_teste['Teste']:
    tt = cleanup(tt)
    prob_tt = 1
    palavra_tt = tt.split()
    for palavra_teste in palavra_tt:
        s = 0 # frequencia da palavra no Indiferente
        p = 0
        if palavra_teste in Freqi:
            s = Freqi[palavra_teste]   #pegando qtas vezes aparece a palavra
        p = (s + alpha)/(pI + palavras_existentes)
        prob_tt = prob_tt * p
    
    prob_fin = prob_tt * prob_category('I')
    Indiferente[tt] = prob_fin
        
Indiferente



{'cs é um bom exemplo do quanto a experiencia internacional faz diferença  lg/sk/mibr tiveram que se mudar do brasil pra atingir o topo  apesar da contínua queda de desempenho  mas ninguém permanece por muito tempo no topo em esporte nenhum ': 2.5696195740708867e-120,
 'acho que o time precisa de uma pausa  tentar por a cabeça no lugar e entender o novo meio de se jogar csgo  pra depois voltarem a jogar juntos  cold viu que o time estagnou e vazou fora pra faze  mibr precisa se reinventar mas precisam estar com a mente no lugar': 3.3443151571970982e-145,
 'mano  o que ta acontecendo com a mibr': 8.2453765943314977e-20,
 '@rafael_loppes1 @mibr @abcfc eu tava acompanhando a fúria  assisti aquela md5 que eles perderam a final para a geng \n\ntaco  fallen  fer e coldzera eram a base do time  cold saiu e só ficou esses 3  acho que se um desses sair o time acaba  hoje tá só o remendo ': 1.6146594252165114e-142,
 'mibr decepcionou hoje  / pelo menos to feliz por motivos óbvios': 2.33701001240

In [180]:
tt_clas = []
tt = 0

for tt in dados_teste['Teste']:
    tt = cleanup(tt)
    palavra_tt = tt.split()
    
    maior = Indiferente[tt]
    if maior < Negativo[tt]:
        maior = Negativo[tt]
    elif maior < Positivo[tt]:
        maior = Positivo[tt]    

        
    if maior == Positivo[tt]:    
        tt_clas.append('P')
    elif maior == Negativo[tt]:    
        tt_clas.append('N')
    elif maior == Indiferente[tt]:    
        tt_clas.append('I')


tt_clas


['I',
 'I',
 'I',
 'I',
 'N',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'N',
 'N',
 'I',
 'I',
 'N',
 'I',
 'N',
 'I',
 'I',
 'N',
 'I',
 'I',
 'N',
 'I',
 'I',
 'I',
 'I',
 'N',
 'I',
 'I',
 'I',
 'N',
 'I',
 'I',
 'N',
 'N',
 'I',
 'P',
 'I',
 'I',
 'I',
 'N',
 'I',
 'I',
 'N',
 'I',
 'I',
 'I',
 'N',
 'I',
 'I',
 'P',
 'I',
 'I',
 'I',
 'I',
 'P',
 'I',
 'I',
 'I',
 'I',
 'P',
 'I',
 'N',
 'N',
 'I',
 'N',
 'I',
 'N',
 'I',
 'N',
 'N',
 'N',
 'I',
 'I',
 'I',
 'N',
 'I',
 'I',
 'I',
 'I',
 'N',
 'N',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'N',
 'N',
 'I',
 'N',
 'N',
 'I',
 'I',
 'I',
 'P',
 'N',
 'P',
 'I',
 'N',
 'I',
 'N',
 'I',
 'N',
 'I',
 'I',
 'I',
 'I',
 'I',
 'N',
 'N',
 'I',
 'I',
 'N',
 'N',
 'P',
 'I',
 'N',
 'I',
 'I',
 'I',
 'N',
 'I',
 'I',
 'N',
 'I',
 'I',
 'I',
 'I',
 'N',
 'I',
 'I',
 'I',
 'I',
 'N',
 'N',
 'I',
 'I',
 'I',
 'P',
 'N',
 'I',
 'N',
 'N',
 'I',
 'I',
 'I',
 'N',
 'I',
 'N',
 'N',
 'N',
 'I',
 'I',
 'I'

In [181]:
dados_teste['Previsto'] = tt_clas
dados_teste

,Classificação,Teste,Previsto
0,P,cs é um bom exemplo do quanto a experiencia in...,I
1,I,"acho que o time precisa de uma pausa, tentar p...",I
2,N,"mano, o que ta acontecendo com a mibr",I
3,N,@rafael_loppes1 @mibr @abcfc eu tava acompanha...,I
4,N,mibr decepcionou hoje :/ pelo menos to feliz p...,N
...,...,...,...
313,N,"caralho mano, \nn vim criticar, mas pqp\na mib...",N
314,I,@fly_vitor11 @smooyacs @mibr congrats o caralh...,N
315,I,"@mibr taco e mey, corram enquanto é tempo, por...",I
316,N,@bida @mibr @flashpoint @chaosec ex time em at...,I


### <font color='#005675'>Acurácia</font>

In [170]:
P = 0
falso_P = 0

N = 0
falso_N = 0

I = 0
falso_I = 0


total = len(dados_teste["Teste"])


i = 0
while i < len(dados_teste['Classificação']):
    if dados_teste['Classificação'][i] == 'N':
        if dados_teste['Previsto'][i] == 'N':
            N += 1
        else:
            falso_N += 1
        
    if dados_teste['Classificação'][i] == 'P':
        if dados_teste['Previsto'][i] == 'P':
            P += 1
        else:
            falso_P += 1
              
    if dados_teste['Classificação'][i] == 'I':
        if dados_teste['Previsto'][i] == 'I':
            I += 1
        else:
            falso_I += 1
    i += 1
    
print(I/total)
print(N/total)
print(P/total)
print(' ')
print(falso_I/total)
print(falso_N/total)
print(falso_P/total)

0.36163522012578614
0.16037735849056603
0.02830188679245283
 
0.1540880503144654
0.21069182389937108
0.08490566037735849


In [171]:
print("Positivos Verdadeiros: ", round((P/total)*100,2), "%")
print("Positivos Falsos: ", round((falso_P/total)*100, 2), "%")
print("Negativos Verdadeiros: ", round((N/total)*100,2), "%")
print("Negativos Falsos: ", round((falso_N/total)*100, 2), "%")
print("Indiferente Verdadeiros: ", round((I/total)*100,2), "%")
print("Indiferente Falsos: ", round((falso_I/total)*100, 2), "%")
print("_"*35)

print("Acertos: ", round(((P/total)+(N/total)+(I/total))*100,2), "%")
print("Erros: ", round(((falso_P/total)+(falso_N/total)+(falso_I/total))*100,2), "%")

Positivos Verdadeiros:  2.83 %
Positivos Falsos:  8.49 %
Negativos Verdadeiros:  16.04 %
Negativos Falsos:  21.07 %
Indiferente Verdadeiros:  36.16 %
Indiferente Falsos:  15.41 %
___________________________________
Acertos:  55.03 %
Erros:  44.97 %


In [187]:
dados_teste.head(20) #P/ ter uma ideia se classificou msm

,Classificação,Teste,Previsto
0,P,cs é um bom exemplo do quanto a experiencia in...,I
1,I,"acho que o time precisa de uma pausa, tentar p...",I
2,N,"mano, o que ta acontecendo com a mibr",I
3,N,@rafael_loppes1 @mibr @abcfc eu tava acompanha...,I
4,N,mibr decepcionou hoje :/ pelo menos to feliz p...,N
5,I,"rt @bielmxrtins: parece ironia, torcer pra mib...",I
6,P,rt @f0restdaking: @mibr @chaosec @envy jogaram...,I
7,I,"@smooyacs @mibr @sick_cs eai smooya, joguei no...",I
8,N,@lantgabriel @fallencs @smooyacs @chaosec @env...,I
9,I,quando a mibr joga?,I



### Por que não se pode utilizar o próprio classificador para gerar mais amostras de treinamento?

Utilizando-se o próprio classificador para gerar mais amostras de treinamento, ele pode acabar ficando enviazado, 'viciado'. Dessa forma, os resultados obtidos tem maiores chances de estarem incorretos, com a análise prejudicada e perdendo a confiabilidade.
Uma forma de aprimorar os resultados e aumentar a confiabilidade deles é puxar mais tweets para classificar, ou seja, ampliar o espaço amostral sem enviezar o classificador.

### Outras limpezas e tranformações:

Como sujestão do grupo para limpeza, coloca-se em foco caracteres que estão presentes em tweets idependentemente das classificações adotadas e que são de fácil localização, como as listadas:

- Trocar vírgulas dos tweets por espaços
- Remover risadas, tais como "kkkk"
- Remover as URLs, como feito na função `clean_extra()`



### diferentes cenários para Naïve Bayes fora do contexto do projeto:

- Analisar reações dos espectadores de determinado filme relevante (um indicado ou ganhador do Oscar, por exemplo), usando classificações como comentários positivos, negativos, irrelevantes;

- Observar reações a políticos de eleitores, havendo classificações como posicionamentos relacionados a diferentes segmentos sociais, tais como relacionados a saúde, educação e economia;

___
## Concluindo

Com os resultados obtidos, tem-se que a acurácia é de 55.03 %, percentual este baixo, haja vista a melhor confiabilidade, por exemplo, de uma classificação com acurácia cujos acertos sejam de pelo menos em torno de 3/2. 

O que faz com que o classificador não seja tão preciso são frases dos usuários do tweeter que se expressam por meio de ironias e sarcasmo, usando palavras, por exemplo, de caráter positivo em um contexto de reprovação do time MIBR.
Ademais, o uso de expressões, tais como risadas, que aparecem independentemente do contexto, podem influenciar e viciar o classificador.

Por fim, um aprofundamento na análise e a obtenção de melhores resultados, poderiam ser feitos aperfeiçoamentos. Um exemplo e uma nova filtragem com objetivo de tirar outros caracteres indesejados que poluem os tweets e selecionar palavras de mesmo significado, escritas de forma diferente e substituí-las por uma padrão. Deste modo, não haverá tanta variação de termos com um mesmo significado, de modo a deixar os resultados mais precisos. Por exemplo, termos referentes ao jogo (Counter Strike), que possui diferentes formas ao ser citado pelos internautas, tais como csgo.

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**